In [ ]:
import pandas as pd

podaci_df = pd.read_csv('/home/pc/Desktop/Istrazivanje-podataka-2/Konzistentnost/spojeni_rezultati_cisti.csv')

# Funkcija za izračunavanje molskih frakcija i frakcijske razlike
def izracunaj_molske_frakcije_i_razlike(sekvenca_df):
    ukupno_aminokiselina = len(sekvenca_df)  # Ukupan broj aminokiselina u sekvenci
    
    # Broj svake aminokiseline u sekvenci
    broj_aminokiselina = sekvenca_df['Aminokiselina'].value_counts()

    # Molske frakcije za svaku aminokiselinu
    molska_frakcija = broj_aminokiselina / ukupno_aminokiselina
    
    # Broj uredjenih i neuredjenih tačaka za ISUNSTRUCT
    broj_uredjenih_isunstruct = sekvenca_df['ISUNSTRUCT_Klasifikacija'].sum()
    broj_neuredjenih_isunstruct = ukupno_aminokiselina - broj_uredjenih_isunstruct
    
    # Molska frakcija za uredjeni deo
    molska_frakcija_isunstruct_uredjeni = broj_uredjenih_isunstruct / ukupno_aminokiselina
    
    # Molska frakcija za neuredjeni deo
    molska_frakcija_isunstruct_neuredjeni = broj_neuredjenih_isunstruct / ukupno_aminokiselina
    
    # Frakcijska razlika prema novoj formuli bez uslova
    frakcijska_razlika_isunstruct = (molska_frakcija_isunstruct_neuredjeni - molska_frakcija_isunstruct_uredjeni) / molska_frakcija_isunstruct_uredjeni
    
    # Procenat molske frakcije uredjenog dela
    procenat_isunstruct = round(molska_frakcija_isunstruct_uredjeni * 100, 2)
    
    # Procenat molske frakcije neuredjenog dela
    procenat_neuredjeni = round(molska_frakcija_isunstruct_neuredjeni * 100, 2)
    
    return molske_frakcije, molska_frakcija_isunstruct_uredjeni, molska_frakcija_isunstruct_neuredjeni, frakcijska_razlika_isunstruct, procenat_isunstruct, procenat_neuredjeni

# Lista za skladištenje rezultata
rezultati = []

# Iteriramo kroz sve fajlove u 'Ime_fajla' koloni
for naziv_fajla in podaci_df['Ime fajla'].unique():
    # Filtriramo podatke za sekvencu iz trenutnog fajla
    sekvenca_df = podaci_df[podaci_df['Ime fajla'] == naziv_fajla]
    
    # Izračunavamo molsku frakciju, procenat i frakcijsku razliku
    molske_frakcije, molska_frakcija_uredjeni, molska_frakcija_neuredjeni, frakcijska_razlika, procenat_uredjeni, procenat_neuredjeni = izracunaj_molske_frakcije_i_razlike(sekvenca_df)
    
    # Dodajemo rezultate u listu
    for aminokiselina, molska_frakcija in molske_frakcije.items():
        rezultati.append({
            'Ime_fajla': naziv_fajla,
            'Aminokiselina': aminokiselina,
            'Molska Frakcija': molska_frakcija,
            'Procenat': f"{round(molska_frakcija * 100, 2)}%",  
            'Frakcijska Razlika': round(frakcijska_razlika, 4),
            'Molska Frakcija Uredjeni': molska_frakcija_uredjeni,
            'Molska Frakcija Neuredjeni': molska_frakcija_neuredjeni,
            'Procenat_uredjeni': f"{procenat_uredjeni}%", 
            'Procenat_neuredjeni': f"{procenat_neuredjeni}%" 
        })

# Kreiramo DataFrame sa rezultatima
rezultati_df = pd.DataFrame(rezultati)

# Čuvanje novog CSV fajla sa rezultatima
rezultati_df.to_csv('rezultati_molske_frakcije_i_frakcijske_razlike.csv', index=False)

# Ispis rezultata
print(rezultati_df.head())


Ako je frakcijska razlika negativna, to znači da je molska frakcija aminokiselina u uredjenom delu veća nego u neuredjenom delu sekvence. Ovo može ukazivati na to da su aminokiseline u uredjenim regionima (koji su obično stabilniji) koncentrisanije, dok u neuredjenim regionima (koji su fleksibilniji) aminokiseline mogu biti prisutne u manjoj koncentraciji. Takva situacija može biti indikacija jače strukturne stabilnosti uredjenih regija u odnosu na neuredjene.
